In [1]:
import re
import numpy as np
import pandas as pd
from natasha import Segmenter, NewsEmbedding, NewsMorphTagger, MorphVocab, Doc
import nltk
from nltk.corpus import stopwords

import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel

/home/o13g/bootcamp/ds-phase-2/rec-project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/o13g/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
morph_vocab = MorphVocab()

/home/o13g/bootcamp/ds-phase-2/rec-project/.venv/lib/python3.12/site-packages/pymorphy2/analyzer.py:114: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [4]:
df = pd.read_csv('../clear_data/genres_clear.csv')

In [5]:
df.head()

,author,title,annotation,img_url,book_url,genre
0,Miroshnik Mariya,Обещание Малфою,"Серый дым клубами поднимался все выше, постепе...",https://rust.litnet.com/uploads/covers/120/161...,https://litnet.com/ru/book/obeshchanie-malfoyu...,"Фанфик, Фанфики по фильмам"
1,Антонина Аркадьева,Любовь На Эгейском Море _aŞk Sana Benzer,"Лето, теплое море и любовь простых красивых ду...",https://rust.litnet.com/uploads/covers/120/159...,https://litnet.com/ru/book/obeshchanie-malfoyu...,"Фанфик, Фанфики по фильмам"
2,Этранжера,"Всё началось с поцелуя. (по сериалу ""Любовь на...","Омер и Дефне, знающий себе цену мужчина и неоп...",https://rust.litnet.com/uploads/covers/120/158...,https://litnet.com/ru/book/obeshchanie-malfoyu...,"Фанфик, Фанфики по фильмам"
3,Люба,Белла,Попаданка в тело Изабеллы Свон....,https://rust.litnet.com/uploads/covers/120/158...,https://litnet.com/ru/book/obeshchanie-malfoyu...,"Фанфик, Фанфики по фильмам"
4,Cool blue lady,Мальчик голубых кровей. Испытание дружбой,Драко Малфой опускает голову и руки его трясут...,https://rust.litnet.com/uploads/covers/120/154...,https://litnet.com/ru/book/obeshchanie-malfoyu...,"Фанфик, Фанфики по фильмам"


In [6]:
stop = set(stopwords.words('russian')) - {'не', 'ни'}
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')

In [7]:
def clean_string(text):
    text = text.lower()
    text = re.sub(r'\s{1}[а-яё]*\.\.\.$', '', text) #remove bad ending
    text = re.sub(r'<.*?>', ' ', text) # removing html-tags from text
    text = re.sub(r'http\S+|\S+@\S+', ' ', text) # removing links
    text = re.sub(r'[^а-яё\s]', ' ', text.lower()) # remove all non-letter symbols
    cleaned_text = re.sub(r'\s+', ' ', text).strip() # remove double or more spaces
    
    return cleaned_text

In [8]:
def lemmantize_words(text):
    lemmas = []
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    
    try:
        for token in doc.tokens:
            token.lemmatize(morph_vocab)
            lemmas.append(token.lemma)
                
    except Exception as e:
        print(f"Ошибка при обработке текста: {text[:30]}... → {e}")
        return ''
    
    return ' '.join(lemmas)

In [9]:
def filter_stop_words(text):
    words = []
    for word in text.split():
        if word in stop or len(word) < 2:
            continue
        words.append(word)
        
    return ' '.join(words)

In [10]:
def preprocess_string(string):
    if not isinstance(string, str) or len(string.strip()) == 0: # fool-protection
        return ''
    
    string = clean_string(string)
    string = lemmantize_words(string)
    string = filter_stop_words(string)
    
    return string

In [11]:
def preprocess_title(string):
    if not isinstance(string, str) or len(string.strip()) == 0: # fool-protection
        return ''

In [12]:
df['annotation'][0]

'Серый дым клубами поднимался все выше, постепенно растворяясь в воздухе. Резкий ожог на пальце напомнил девушке, сидящей на подоконнике, что сигарета уже давно дог...'

In [13]:
preprocess_string(df['annotation'][0])

'серый дым клуб подниматься высоко постепенно растворяться воздух резкий ожог палец напомнить девушка сидеть подоконник сигарета давно'

In [14]:
df.shape, df.dropna(subset='genre').shape

((67557, 6), (67553, 6))

In [15]:
df.dropna(subset='genre', inplace=True)

In [16]:
df['clean_annotation'] = df['annotation'].apply(preprocess_string)

In [17]:
df['clean_title'] = df['title'].apply(clean_string)

In [18]:
df['clean_genre'] = df['genre'].apply(clean_string)

In [20]:
df

,author,title,annotation,img_url,book_url,genre,clean_annotation,clean_title,clean_genre
0,Miroshnik Mariya,Обещание Малфою,"Серый дым клубами поднимался все выше, постепе...",https://rust.litnet.com/uploads/covers/120/161...,https://litnet.com/ru/book/obeshchanie-malfoyu...,"Фанфик, Фанфики по фильмам",серый дым клуб подниматься высоко постепенно р...,обещание малфою,фанфик фанфики по фильмам
1,Антонина Аркадьева,Любовь На Эгейском Море _aŞk Sana Benzer,"Лето, теплое море и любовь простых красивых ду...",https://rust.litnet.com/uploads/covers/120/159...,https://litnet.com/ru/book/obeshchanie-malfoyu...,"Фанфик, Фанфики по фильмам",лето теплый море любовь простой красивый душа ...,любовь на эгейском море,фанфик фанфики по фильмам
2,Этранжера,"Всё началось с поцелуя. (по сериалу ""Любовь на...","Омер и Дефне, знающий себе цену мужчина и неоп...",https://rust.litnet.com/uploads/covers/120/158...,https://litnet.com/ru/book/obeshchanie-malfoyu...,"Фанфик, Фанфики по фильмам",омер дефне знать цена мужчина неопытный девушк...,всё началось с поцелуя по сериалу любовь напрокат,фанфик фанфики по фильмам
3,Люба,Белла,Попаданка в тело Изабеллы Свон....,https://rust.litnet.com/uploads/covers/120/158...,https://litnet.com/ru/book/obeshchanie-malfoyu...,"Фанфик, Фанфики по фильмам",попаданка тело изабелла свон,белла,фанфик фанфики по фильмам
4,Cool blue lady,Мальчик голубых кровей. Испытание дружбой,Драко Малфой опускает голову и руки его трясут...,https://rust.litnet.com/uploads/covers/120/154...,https://litnet.com/ru/book/obeshchanie-malfoyu...,"Фанфик, Фанфики по фильмам",драко малфа опускать голова рука трястись подн...,мальчик голубых кровей испытание дружбой,фанфик фанфики по фильмам
...,...,...,...,...,...,...,...,...,...
67552,Михаил Петров,Дарли-сити. Родом из Дарли,Сложное и в то же время простое обстоятельство...,https://rust.litnet.com/uploads/covers/120/157...,https://litnet.com/ru/book/richard-kasl-cikl-z...,"Детективы, Полицейский детектив",сложный время простой обстоятельство случаться...,дарли сити родом из дарли,детективы полицейский детектив
67553,Юлия Лисова,Всё останется загадкой..,"Что делать, когда встретился с тем, что можешь...",https://rust.litnet.com/uploads/covers/120/154...,https://litnet.com/ru/book/richard-kasl-cikl-z...,"Детективы, Полицейский детектив",делать встретиться мочь объяснить никто не пов...,всё останется загадкой,детективы полицейский детектив
67554,Катя Литвин,"Уснул, называется","Вот смотрю, бывает, кино про полицейских и дум...",https://rust.litnet.com/uploads/covers/120/174...,https://litnet.com/ru/book/richard-kasl-cikl-z...,"Детективы, Полицейский детектив",смотреть бывать кино полицейский думать откуда...,уснул называется,детективы полицейский детектив
67555,Кристина Выборнова (Аделя Хильман),Тайны Колина,Очередной рассказ-фрагмент из цикла о жизни Ко...,https://rust.litnet.com/uploads/covers/120/150...,https://litnet.com/ru/book/richard-kasl-cikl-z...,"Детективы, Полицейский детектив",очередной рассказ фрагмент цикл жизнь колин ро...,тайны колина,детективы полицейский детектив


In [39]:
def concat_clean_content(row):
    return (
    "Жанр: " + row['genre'] + "\n" +
    "Аннотация: " + row['annotation'] + "\n" +
    "Название: " + row['title']
)

In [40]:
df['clean_concat_content'] = df.apply(concat_clean_content, axis=1)[0]

In [42]:
df.to_csv('final.csv', index=False)